### Preprocess Raw-data

start env preprocess

#### import packages 

In [ ]:
import glob
import os
import h5netcdf
import xarray as xr
import numpy as np
import pandas as pd
import rioxarray as rio
from geopandas import read_file
from tqdm.notebook import tqdm
from preprocess.uaUtils import open_fua
from preprocess.s2Utils import search_product,open_tile,clip_tile, get_tile_info, get_prod_info
from preprocess.utils import rasterize,get_obj_within,str_to_oao, save_cube,class_dict,org_files
long_name=('B4 (665 nm)',
 'B3 (560 nm)',
 'B2 (490 nm)',
 'B8 (842 nm)',
 'SRB5 (705 nm)',
 'SRB6 (740 nm)',
 'SRB7 (783 nm)',
 'SRB8A (865 nm)',
 'SRB11 (1610 nm)',
 'SRB12 (2190 nm)') 
xr.set_options(keep_attrs=True)

### User input

define where the raw-data is located (selected Sentinel tiles, patches/AOI, and LULC-Data)
and define where to save the data. 

In [ ]:

#  Define root_path containing: 
# "raw-data/AOI/tile_selection.gpkg", 
# 'raw-data/AOI/patch_size_features.gpkg', 
# 'raw-data/LULC-Sweden/*/Data/*.gpkg'

root_path = os.getcwd()
timeperiod = 1 # timeperiod 
patch_size = 256  #choose between ['64','128','256'] 

# Path to s2 products
##s2_path = os.path.join(root_path,'raw-data/sentinel-2/timeperiod{}/'.format(timeperiod))
s2_path = os.path.join(root_path,'../dsen2_results/timeperiod{}/'.format(timeperiod))

# Path to Urban Atlas (UA) data
ua_paths = glob.glob(os.path.join(root_path,'raw-data/LULC-Sweden/*/Data/*.gpkg'))

# Define where to save processed data
source = 'dsen2'
data_source = source + '_{}_new'.format(patch_size)
sdir = os.path.join(root_path,'processed-data/{source}/timeperiod{tp}'.format(source=data_source,tp=timeperiod))

#### Check if products exists. 

In [ ]:
print('Looking for sentinel products in:\n',s2_path,'\n')

# get crs
ua_crs = read_file(ua_paths[0]).crs

# path to s2 tiles to use 
s2tiles = read_file(os.path.join(root_path,'raw-data/AOI/tile_selection.gpkg')).to_crs(ua_crs)

# path to scb patches of size*size
scb_grid = read_file(os.path.join(root_path,'raw-data/AOI/patch_size_features.gpkg'),layer = "patch_{}".format(patch_size)).to_crs(ua_crs)


# check that desired s2tiles existst in s2_path
print('---- Found sentinel products: ----')
for tile in s2tiles.itertuples():
    print(tile.Name)
    prod,date= search_product(s2_path,tile.Name)
    if prod:
        for i, prod in enumerate(prod):
            fn=os.path.basename(prod)
            print('Found:',get_prod_info(fn)['Tile'],pd.to_datetime(date[i]),fn)
    else:
        print('No found sentinel products for tile:',tile.Name)
          
# list found FUAS
print('\n---- Found LULC data: ----')
if ua_paths:
    for ua_path in ua_paths:
        print('Found:',os.path.basename(ua_path))
else:
    print('No LULC data found')
    

print('\n Save processed data to: \n',  sdir)

In [ ]:
# sequential all fuas (original)
bad_patch=[]
for path in tqdm(ua_paths, desc='Total progress'):
    
    fua_labls,fua_bound = open_fua(path)             # open fua layers
    tiles_inters = s2tiles.sjoin(fua_bound, predicate='intersects')  #get intersecting tiles for fua
    fua_name = str_to_oao(fua_bound.fua_name[0])
    
    for tile in tqdm(tiles_inters.itertuples(),desc='Intersecting tiles',total=len(tiles_inters)): # for every intersecting tile
        
        # select patches within curr tile
        curr_tile = s2tiles[s2tiles.Name.isin([tile.Name])]                                               
        patches_within = get_obj_within(get_obj_within(scb_grid,curr_tile),fua_bound) #select patches(grid) within curr_tile and within fua
        #savedir
        savedir = os.path.join(sdir,'{}/{}/'.format(fua_name,tile.Name))
        #print(savedir)
        #open curr tile 
        s2_tile = open_tile(tile.Name,s2_path)
    
        for patch in tqdm(patches_within.itertuples(),total=len(patches_within), desc='FUA: {}, Tile: {}'.format(fua_name,tile.Name)):
            
            # get current patch and clip labels to patch extent
            curr_patch = scb_grid[(scb_grid.id.isin([patch.id]))]
            # clip tile and labels to patch extent
            s2_patch = clip_tile(s2_tile,curr_patch)
            s2_patch = s2_patch.astype(np.float32)
            patch_labls = fua_labls.clip(curr_patch)
            #rasterize patch labels 
            try:
                cube = rasterize(patch_labls)
            except:
                bad_patch.append(patch.id)
            else:    
                #merge s2_patch and patch labels
                cube = cube.merge(s2_patch.rio.reproject_match(cube)) #append to cube
                # reduce coordinate dimensions by one
                if cube.dims['x'] > patch_size:
                    cube=cube.isel(x=slice(None, -1), y=slice(None, -1))           
            
                #set attributes to cube before saving
                cube.attrs["patch_id"] = patch.id
                cube.attrs['FUA'] = fua_name
                cube.attrs['long_name']=long_name
                for key,value in get_tile_info(tile.Name,s2_path).items():
                    cube.attrs[key] = value
                #cube.train_id.attrs['_fillValue']=255
                #cube.class_code.attrs['_fillValue']=255
                save_cube(cube,savedir)

### Remove bad patches 
Execute this step for once for each timeperiod. 

In [ ]:
# calculate min max to check for bad data

from tqdm.notebook import tqdm
from dataset.datasets import s2stats, sentinel
#import torch
import numpy as np
import os

dset= s2stats(root_dir='processed-data/dsen2_256_new/timeperiod1/*/*')
#dset= sentinel(root_dir='processed-data/dsen2_256_new_split/',timeperiod=2)
print(len(dset))
max_vals=np.zeros((len(dset),10))
min_vals=np.zeros((len(dset),10))

count = 0
paths = []

for idx, (img,path) in enumerate(dset):
   
    maxs = img.numpy().max(axis=(1,2))
    mins = img.numpy().min(axis=(1,2))
    
    if any(maxs >3.4e+38):
        count +=1
        paths.append(path)
    
    max_vals[idx,:]=img.numpy().max(axis=(1,2))
    min_vals[idx,:]=img.numpy().min(axis=(1,2))
    
maxs= max_vals.max(axis=0)
mins= min_vals.min(axis=0)

#print(maxs)
#print(mins)


for path in paths: 
    os.remove(path)

print('{} bad patches are removed.'.format(count))

### split train/test/val

when AOI/patches for timeperiod 1 and 2 are preprocessed 

1. Split dataset (if satisfied, execute step 2).
2. Reorganize files after split.

#### step 1

In [ ]:

# find_tp2_patches()
# takes in a list of paths to patches in timeperiod 1, 
# finds and returns corresponding patch by patch id in timeperiod_2
def find_tp2_patches(df):
    df_2 = []
    for path in df:
        patch = os.path.split(path)[-1].split('_')
        patch_id='_'.join(patch[0:3])
        df_2.append(glob.glob(os.path.join('processed-data/dsen_2_256_new/timeperiod2/*/*',patch_id+'_*'))[0])
    return(df_2)

#1 split dataset
import glob
import os
train = 0.8
test = 0.5 #of remaining -train
sdir='/home/exjobbare/projects/datadrive0/deepSat/processed-data/dsen_2_256_new/timeperiod1/'
df = np.asarray(glob.glob(os.path.join(sdir,'*/*/*.nc')))
total = len(df)
msk = np.random.rand(len(df)) <train
len(msk[msk==True])/len(msk)

train = df[msk]
train_2 = find_tp2_patches(train)
testval = df[~msk]
msk = np.random.rand(len(testval))<test
test=testval[msk]
test_2=find_tp2_patches(test)
val = testval[~msk]
val_2=find_tp2_patches(val)

print('total:',total,'train:', len(train) ,round(len(train)/len(df)*100,0),'test:',len(test),round(len(test)/len(df)*100,0),'val:',len(val),round(len(val)/len(df)*100,0))

#### step 2

In [ ]:
# step 2 copy files to 'source_split' '/train' '/test' or '/val'
if train.size>0: 
    org_files(train,mode='train')
    org_files(train_2,mode='train')
if test.size>0:
    org_files(test,mode='test')
    org_files(test_2,mode='test')
if val.size>0:
    org_files(val,mode='val')
    org_files(val_2,mode='val')


#### Compact Naming Convention

The compact naming convention is arranged as follows:

MMM_MSIXXX_YYYYMMDDHHMMSS_Nxxyy_ROOO_Txxxxx_<Product Discriminator>.SAFE

The products contain two dates.

The first date (YYYYMMDDHHMMSS) is the datatake sensing time.
The second date is the "Product Discriminator" field, which is 15 characters in length, and is used to distinguish between different end user products from the same datatake. Depending on the instance, the time in this field can be earlier or slightly later than the datatake sensing time.

The other components of the filename are:

* MMM: is the mission ID(S2A/S2B)
* MSIXXX: MSIL1C denotes the Level-1C product level/ MSIL2A denotes the Level-2A product level
* YYYYMMDDHHMMSS: the datatake sensing start time
* Nxxyy: the PDGS Processing Baseline number (e.g. N0204)
* ROOO: Relative Orbit number (R001 - R143)
* Txxxxx: Tile Number field

SAFE: Product Format (Standard Archive Format for Europe)

Source https://sentinel.esa.int/web/sentinel/user-guides/sentinel-2-msi/naming-convention

In [ ]:
bad_p = ['46008',
 '48950',
 '47261',
 '45171',
 '45997',
 '45592',
 '47692',
 '41799',
 '48950',
 '48514',
 '48514',
 '60278',
 '60679',
 '56922',
 '53536',
 '56077',
 '58600',
 '59854',
 '59009',
 '56053',
 '51452',
 '9139',
 '8715',
 '7450',
 '4934',
 '5354',
 '4082',
 '5349',
 '4923',
 '11652',
 '8299',
 '9551',
 '10387',
 '8719',
 '7050',
 '9139',
 '8719',
 '7470',
 '8299',
 '60303',
 '64920',
 '56945',
 '62823',
 '59454',
 '59460',
 '57356',
 '65340',
 '60731',
 '60723',
 '61151',
 '59457',
 '60311',
 '54844',
 '59891',
 '62403',
 '61143',
 '59880',
 '59454',
 '59871',
 '58607',
 '63229',
 '63226',
 '57346',
 '60288',
 '63640',
 '62809',
 '59024',
 '16274',
 '15017',
 '15430',
 '13760',
 '15017',
 '45635',
 '42276',
 '42276',
 '47310',
 '45635',
 '15505',
 '16766',
 '15087',
 '26785',
 '26365',
 '28035',
 '28455',
 '28867',
 '28455',
 '28035',
 '33010',
 '31752',
 '37211',
 '35105',
 '34262',
 '38048',
 '41027',
 '40601',
 '43553',
 '41035',
 '38923',
 '35984',
 '40604',
 '39349',
 '18468',
 '19313',
 '20985',
 '18041',
 '17213',
 '17200',
 '19306',
 '19730',
 '18896']
